# 处理输出核素列表（排序）

In [26]:
# V 2.0
# 对数据进行处理，将Geant4直接输出的核素进行归纳整理
# # 过滤掉 ignoreList = ['stable','fs','ps','ns','us','eV','+'] 的核素
# # 合并同质异能素
# 输出两个表：1、按数量排序，从多到少；2、按衰变时间排序，从慢到快
import os,sys 
os.chdir(sys.path[0])
os.chdir('../build/output')

import re 
patNum = '\d.*? '
patLet = '[a-zA-Z]+'

unitList = {
    "fs": 1e-12,
    "ns": 1e-9,
    "us": 1e-6,
    "ms": 1e-3,
    "s": 1,
    "min": 60,
    "h": 3600,
    "d": 24*3600,
    "m": 30*24*3600,
    "y": 365*24*3600
}

def getUnitCoff(unit):
    return unitList.get(unit, "Invalid Unit")

for eachfile in os.listdir():
    if "Simple" in eachfile: # 跳过简单的数据列表
        continue
    if "processed" in eachfile: # 跳过处理过的数据列表
        continue
    
    with open(eachfile,'r') as f:
        data = f.readlines()[2:]
        print(eachfile)

    nuclied = dict()
    ignoreList = ['stable','fs','ps','ns','us','eV','+']
    for aux in data:
        aux = aux.split("\t")
        for igList in ignoreList: # 跳过稳定、超短寿命的核素不做统计
            if igList in aux[-1].split("\n")[0]:
                isJump = 1
        if isJump:
            isJump = 0
            continue
        
        val = float(re.findall(patNum,aux[-1].split("\n")[0])[0])
        unit =  re.findall(patLet,aux[-1].split("\n")[0])[0]
        trueLife = val * getUnitCoff(unit) # 以秒为单位的半衰期
        try:
            nuclied[aux[0].split("[")[0]] = [int(nuclied[aux[0].split("[")[0]][0])+int(aux[1]),val,unit,trueLife]
        except: # 如果报错的话，说明对nuclied索引错误，因为还没有建立该条目
            try: # 那么建立该条目
                nuclied[aux[0].split("[")[0]] = [int(aux[1]),val,unit,trueLife]
            except:
                # 跳过读取的文件行"---------- run 0 -----------"
                # 这样可能有问题，因为多次写入的文档不一定是我们想要的
                continue 

    with open("processed_"+eachfile, 'w') as file:
        for item in sorted(nuclied.items(),key = lambda x:x[1][0],reverse = True):
            # print(item[0],item[1][0],item[1][1],item[1][2])
            dataString = "%s,%s,%.2f,%s\n" %(item[0],item[1][0],item[1][1],item[1][2])
            print(dataString)
            file.write(dataString)
        file.write("\n")
        for item in sorted(nuclied.items(),key = lambda x:x[1][3]):
            # print(item[0],item[1][0],item[1][1],item[1][2])
            dataString = "%s,%s,%.2f,%s\n" %(item[0],item[1][0],item[1][1],item[1][2])
            print(dataString)
            file.write(dataString)



Ti75.csv
neutron,120887,14.67,min

Ti45,2847,4.44,h

Sc47,2680,4.83,d

Sc44,2164,3.52,d

Sc46,1699,27.05,s

Ti44,1588,85.32,y

triton,1361,12.32,y

V48,1129,23.04,d

Ca41,961,147256.00,y

V47,805,47.03,min

Sc43,795,5.61,h

Ca45,706,234.60,d

Ar37,306,50.51,d

V49,300,1.30,y

Sc48,285,2.63,d

Ar39,217,388.35,y

K42,202,17.83,h

Sc49,151,1.37,h

Cl36,131,434982.00,y

K43,125,1.34,d

S35,88,126.05,d

V46,79,609.54,ms

P32,78,20.58,d

P33,67,36.57,d

Ar42,42,47.50,y

Ar41,39,2.64,h

Fe55,36,3.96,y

Ca47,35,6.54,d

K44,35,31.93,min

Sc42,33,982.91,ms

K38,28,1.33,s

Si31,26,3.78,h

Na24,25,29.11,ms

Si32,21,220.88,y

Al28,20,3.24,min

Cl38,18,53.73,min

Na22,15,3.76,y

C11,14,29.38,min

P30,14,3.60,min

Cl39,12,1.35,h

K45,11,25.69,min

Ti43,10,734.33,ms

F18,8,2.64,h

F20,8,15.97,s

Al29,7,9.46,min

Ar35,7,2.56,s

Ca39,7,1.24,s

Co55,7,1.05,d

He6,7,1.16,s

O15,7,2.94,min

S31,7,3.68,s

Be7,6,76.78,d

Cl34,6,2.20,s

Cr51,6,39.96,d

Na25,6,1.42,min

P34,6,17.93,s

C14,5,8229.00,y

Fe53,5,1

# 处理输出数据，进行符合分析

In [10]:
import sys,os 
os.chdir(sys.path[0] + "/../build/")

crystalName = "GAGG"
crystalDep = []
outGamma = []

with open("./CoinOutGamma_%s.csv" %crystalName,'r') as f:
    for line in f:
        outGamma.append(list(map(float,line.split("\n")[0].split(","))))

with open("./CoinCrystalDep_%s.csv" %crystalName,'r') as f:
    for line in f:
        crystalDep.append(list(map(float,line.split("\n")[0].split(","))))



In [31]:
'''
description: 
param {*} s_start       开始搜索可以匹配的索引位置
param {*} s_end         结束搜索可以匹配的索引位置，通常是-1
param {*} iUnit         需要对待查询列表的单位进行处理， crystalDep[index][0]/unitVal
param {*} quering       待查询的数据
param {*} crystalDep    沉积能量的 Time-Edep 列表，待搜索的列表
return {*}
'''
def findCloseData(s_start,s_end,iUnit,quering,crystalDep):
    unitList = {
    "fs": 1e-12,
    "ns": 1e-9,
    "us": 1e-6,
    "ms": 1e-3,
    "s": 1,
    "min": 60,
    "h": 3600,
    "d": 24*3600,
    "m": 30*24*3600,
    "y": 365*24*3600
    }

    def getUnitCoff(unit):
        return unitList.get(unit, "Invalid Unit")

    if type(iUnit) == type('s'): # 如果输入的是单位的名称的话
        # print("type(iUnit) == type('s') == True")
        unitVal = getUnitCoff(iUnit)
        if unitVal == "Invalid Unit":
            raise unitVal
    else: # 手动指定需要除以的系数
        unitVal = iUnit 
        # print("type(iUnit) == type('s') != True, unitVal = %s" %unitVal)

    if s_end == -1:
        s_end = len(crystalDep)

    index = s_start
    for data in crystalDep:
        if index >= s_end:
            break
        if crystalDep[index][0]/unitVal > quering: # 我单位弄成10秒了....故这里都手动矫正回来
            # print(crystalDep[index][0]/unitVal , quering)
            break
        index = index + 1

    # if index == len(crystalDep) - 1: # 如果已经查询到最后一个数据了
    if index == 0: # 第一个数据，crystalDep[index-1] 没有意义
        return [index, crystalDep[index],crystalDep[index]]
    elif index == len(crystalDep): # 最后一个数据
        return [index, crystalDep[index-1],crystalDep[index-1]]
    else:
        return [index, crystalDep[index-1],crystalDep[index]]



In [32]:
# coincrystalDepLowList = []
# coincrystalDepUpList = []

iUnit = 1            # 需要进行调整的单位，参考 crystalDep[index][0]/iUnit 
coinThreshold = 1e-8 # 符合分辨的阈值，如果两次事件的时间差距在这个范围内，认为可以符合

s_start = 0
index = 0
with open('coinProcessed.csv', 'w') as f:
    f.writelines("isCoin,OutGammaTime,OutGammaEdep,CoinLowTime,CoinLowEdep,CoinUpTime,CoinUpEdep\n")
    for gamma in outGamma:
        isCoin = 0
        [newStart, coincrystalDepLow,coincrystalDepUp] = findCloseData(s_start,-1,iUnit,gamma[0],crystalDep)
        # coincrystalDepLowList.append(coincrystalDepLow)
        # coincrystalDepUpList.append(coincrystalDepUp)
        if (abs(coincrystalDepLow[0] / iUnit - gamma[0]) < coinThreshold) or \
            (abs(coincrystalDepUp[0] / iUnit - gamma[0]) < coinThreshold):
            isCoin = 1 # 符合上了

        f.writelines("%s,%s,%s,%s,%s,%s,%s\n" %(isCoin, gamma[0], gamma[1],
                                                        coincrystalDepLow[0] / iUnit,coincrystalDepLow[1],
                                                        coincrystalDepUp[0] / iUnit,coincrystalDepUp[1]))
        s_start = newStart
        index = index + 1
        if index % (len(outGamma)//10) == 0:
            # print("%s / 100" %(index // (len(outGamma)//100)))
            print("%s / %s" %(index,len(outGamma)))
        if s_start > len(crystalDep):
            print("out of List")
            break
        if isCoin == 0:
            print("gamma time = %s has no Coinicdence" %gamma[0])



24 / 240
48 / 240
72 / 240
96 / 240
120 / 240
144 / 240
168 / 240
192 / 240
216 / 240
240 / 240


In [28]:
len(crystalDep)

309

In [15]:
print("%s / 100" %(index // (len(outGamma)//100)))

117 / 100


In [26]:
index

234

In [17]:
(len(outGamma)//100)

2

In [4]:
os.listdir()

['cmake_install.cmake',
 'CMakeCache.txt',
 'vis.mac',
 'CoinOutGamma_GAGG.csv',
 'GAGG_Rebin_Full.pdf',
 'GAGG.root',
 'output',
 'myRdecay',
 'GAGG_Full.pdf',
 'GAGG_TimevsEng.csv',
 'test',
 'Makefile',
 'CoinCrystalDep_GAGG.csv',
 'CMakeFiles',
 'GAGG_TimePrj.pdf']